# Calculating the reserve Factors of the panels 

# Please enter your name 

In [1]:
name = "yannis"

# Imports 

In [2]:
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))
sys.path.insert(0, os.path.abspath('../optimization'))

from formulas.panels import *
from formulas.helpers import *
from formulas.abd_matrix import *

 

In [3]:
'''# config parser
import configparser

# get the rounding_digits from the ini file
config = configparser.ConfigParser()
config.read('../config.ini')
rounding_digits = int(config['DEFAULT']['rounding_digits'])'''

"# config parser\nimport configparser\n\n# get the rounding_digits from the ini file\nconfig = configparser.ConfigParser()\nconfig.read('../config.ini')\nrounding_digits = int(config['DEFAULT']['rounding_digits'])"

# Constants (Changed based on your data)

In [4]:
personal_data = personal_data_provider(name)
E_11 = personal_data[0]
E_22 = personal_data[1]
G_12 = personal_data[2]
nu_12 = personal_data[3]
knockdown = 0.9 
print(f"Your personal data is: E11 = {E_11}, E22 = {E_22}, G12 = {G_12}, nu12 = {nu_12}. Please verify.")

# Panel laminate properties 
panelStack = [45,45,-45,-45,0,0,90,90,90,90,0,0,-45,-45,45,45]
tPanel = 0.552 

#Panel constants 
length = 750
width = 400
panelthickness = 8.832


Your personal data is: E11 = 132583.92, E22 = 10198.76, G12 = 5099.38, nu12 = 0.33. Please verify.


# Import needed ABD values 

In [5]:
ABD_panel, ABD_panel_inverse = calculateABD(stacksequence=panelStack, plyT=tPanel, EModulus1=E_11, EModulus2=E_22, ShearModulus=G_12)
ABD_panel = ABD_panel * knockdown
ABD_panel_inverse = ABD_panel_inverse * knockdown

## Extract ABD parameters 


In [6]:
D11 = ABD_panel[3][3]
D22 = ABD_panel[4][4]
D12 = ABD_panel[3][4]
D66 = ABD_panel[5][5]
print(D11, D22, D12, D66)

2698671.142592391 2100821.66410216 1495574.8566103033 1583690.6878573217


# Open stress values & properties of panels 

In [7]:
# New version of importing panel stress file
paneldf = pd.read_csv(f'../data/{name}/panel.csv')
paneldf = paneldf.rename(columns={'Elements':'Element ID', 'XX':'sigmaXX', 'YY':'sigmaYY', 'XY':'sigmaXY', 'Loadcase':'Load Case'})
paneldf['Component Name'] = paneldf.apply(elementComponentMatch, axis=1)
paneldf = paneldf.drop(['FileID', 'Step', 'Layer'], axis=1)
paneldf.head(5)


,Element ID,Load Case,sigmaXX,sigmaXY,sigmaYY,Component Name
0,1,1,-41.655796,16.694225,5.143600,panel1
1,2,1,-50.504929,13.890759,6.055713,panel1
2,3,1,-41.692043,14.728354,13.374496,panel1
3,4,1,-41.826157,15.569081,7.285109,panel1
4,5,1,-38.997730,15.624921,6.635034,panel1


# Split up the loadcases 

In [8]:
loadCase1df = paneldf[paneldf['Load Case'] == 1]
loadCase2df = paneldf[paneldf['Load Case'] == 2]
loadCase3df = paneldf[paneldf['Load Case'] == 3]


#Set index back to Element ID 
loadCase1df = loadCase1df.set_index('Element ID')
loadCase2df = loadCase2df.set_index('Element ID')
loadCase3df = loadCase3df.set_index('Element ID')

#Give some overview 
loadCase1df.head(5)

,Load Case,sigmaXX,sigmaXY,sigmaYY,Component Name
Element ID,,,,,
1,1,-41.655796,16.694225,5.143600,panel1
2,1,-50.504929,13.890759,6.055713,panel1
3,1,-41.692043,14.728354,13.374496,panel1
4,1,-41.826157,15.569081,7.285109,panel1
5,1,-38.997730,15.624921,6.635034,panel1


# Now we need to combine into panels:
-Panel ID (as our ID)
-sigmaXXavg
-sigmaYYavg
-sigmaXYavg
-length
-width
-thickness 


In [9]:
# We can just average the stresses @fabianober Tackle rounding!!!! 
#For load case 1 
panelLC1 = loadCase1df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
})
dataframeLength = len(panelLC1)
panelLC1['length'] = [length]*dataframeLength
panelLC1['width'] = [width]*dataframeLength
panelLC1['thickness'] = [panelthickness] * dataframeLength
panelLC1['Load Case'] = ['LC1'] * dataframeLength


#For load case 2 
panelLC2 = loadCase2df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean'
})
dataframeLength = len(panelLC2)
panelLC2['length'] = [length]*dataframeLength
panelLC2['width'] = [width]*dataframeLength
panelLC2['thickness'] = [panelthickness] * dataframeLength
panelLC2['Load Case'] = ['LC2'] * dataframeLength


#For load case 3
panelLC3 = loadCase3df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean'
})
dataframeLength = len(panelLC3)
panelLC3['length'] = [length]*dataframeLength
panelLC3['width'] = [width]*dataframeLength
panelLC3['thickness'] = [panelthickness] * dataframeLength
panelLC3['Load Case'] = ['LC3'] * dataframeLength


panelLC1

,sigmaXX,sigmaYY,sigmaXY,length,width,thickness,Load Case
Component Name,,,,,,,
panel1,-42.697845,8.136578,15.483408,750,400,8.832,LC1
panel2,-39.443552,8.188004,16.935501,750,400,8.832,LC1
panel3,-39.528903,8.051509,17.415443,750,400,8.832,LC1
panel4,-39.395589,8.160427,16.368667,750,400,8.832,LC1
panel5,-42.484748,8.082001,14.220927,750,400,8.832,LC1


# Now we can apply the functions to the load cases 

In [10]:
panelLC1[['sig_crit_shear', 'sig_crit_biax', 'Reserve Factor']] = panelLC1.apply(panelBuckApply, D11=D11, D12=D12, D22=D22,
                                                                                  D66=D66, axis=1, result_type='expand')
panelLC2[['sig_crit_shear', 'sig_crit_biax', 'Reserve Factor']] = panelLC2.apply(panelBuckApply, D11=D11, D12=D12, D22=D22,
                                                                                  D66=D66, axis=1, result_type='expand')
panelLC3[['sig_crit_shear', 'sig_crit_biax', 'Reserve Factor']] = panelLC3.apply(panelBuckApply, D11=D11, D12=D12, D22=D22,
                                                                                  D66=D66, axis=1, result_type='expand')
panelLC1

,sigmaXX,sigmaYY,sigmaXY,length,width,thickness,Load Case,sig_crit_shear,sig_crit_biax,Reserve Factor
Component Name,,,,,,,,,,
panel1,-42.697845,8.136578,15.483408,750,400,8.832,LC1,108.22899,76.477754,1.131854
panel2,-39.443552,8.188004,16.935501,750,400,8.832,LC1,108.22899,77.031279,1.214829
panel3,-39.528903,8.051509,17.415443,750,400,8.832,LC1,108.22899,76.903741,1.205885
panel4,-39.395589,8.160427,16.368667,750,400,8.832,LC1,108.22899,77.016636,1.221379
panel5,-42.484748,8.082001,14.220927,750,400,8.832,LC1,108.22899,76.467143,1.146475


# Create original output 

In [11]:
#Drop irrelevant columns
panelLC1 = panelLC1.drop(['length', 'width', 'thickness', 'Load Case'], axis=1)
panelLC2 = panelLC2.drop(['length', 'width',  'thickness', 'Load Case'], axis=1)
panelLC3 = panelLC3.drop(['length', 'width',  'thickness', 'Load Case'], axis=1)
panelLC1.head(3)

,sigmaXX,sigmaYY,sigmaXY,sig_crit_shear,sig_crit_biax,Reserve Factor
Component Name,,,,,,
panel1,-42.697845,8.136578,15.483408,108.22899,76.477754,1.131854
panel2,-39.443552,8.188004,16.935501,108.22899,77.031279,1.214829
panel3,-39.528903,8.051509,17.415443,108.22899,76.903741,1.205885


In [12]:
# Rename columns to be unambigous
panelLC1 = panelLC1.rename(columns={"Reserve Factor": "LC 1 RF", 'sigmaXX':'sigmaXXLC1', 'sigmaYY':'sigmaYYLC1', 'sigmaXY':'sigmaXYLC1', 'sig_crit_shear':'sig_crit_shearLC1', 'sig_crit_biax':'sig_crit_biaxLC1'})
panelLC2 = panelLC2.rename(columns={"Reserve Factor": "LC 2 RF", 'sigmaXX':'sigmaXXLC2', 'sigmaYY':'sigmaYYLC2', 'sigmaXY':'sigmaXYLC2', 'sig_crit_shear':'sig_crit_shearLC2', 'sig_crit_biax':'sig_crit_biaxLC2'})
panelLC3 = panelLC3.rename(columns={"Reserve Factor": "LC 3 RF", 'sigmaXX':'sigmaXXLC3', 'sigmaYY':'sigmaYYLC3', 'sigmaXY':'sigmaXYLC3', 'sig_crit_shear':'sig_crit_shearLC3', 'sig_crit_biax':'sig_crit_biaxLC3'})
panelLC1

,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,sig_crit_shearLC1,sig_crit_biaxLC1,LC 1 RF
Component Name,,,,,,
panel1,-42.697845,8.136578,15.483408,108.22899,76.477754,1.131854
panel2,-39.443552,8.188004,16.935501,108.22899,77.031279,1.214829
panel3,-39.528903,8.051509,17.415443,108.22899,76.903741,1.205885
panel4,-39.395589,8.160427,16.368667,108.22899,77.016636,1.221379
panel5,-42.484748,8.082001,14.220927,108.22899,76.467143,1.146475


In [13]:
outputDf = pd.concat([panelLC1, panelLC2, panelLC3], axis=1)
outputDf

,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,sig_crit_shearLC1,sig_crit_biaxLC1,LC 1 RF,sigmaXXLC2,sigmaYYLC2,sigmaXYLC2,sig_crit_shearLC2,sig_crit_biaxLC2,LC 2 RF,sigmaXXLC3,sigmaYYLC3,sigmaXYLC3,sig_crit_shearLC3,sig_crit_biaxLC3,LC 3 RF
Component Name,,,,,,,,,,,,,,,,,,
panel1,-42.697845,8.136578,15.483408,108.22899,76.477754,1.131854,-42.916384,-7.917600,15.456793,108.22899,62.079210,0.923475,-14.302086,2.639952,61.968419,108.22899,76.285332,0.981501
panel2,-39.443552,8.188004,16.935501,108.22899,77.031279,1.214829,-39.430905,-8.217435,17.080579,108.22899,60.976510,0.974634,-12.703550,2.446052,67.486285,108.22899,76.541937,0.889847
panel3,-39.528903,8.051509,17.415443,108.22899,76.903741,1.205885,-39.804861,-8.170506,17.327509,108.22899,61.118929,0.966580,-13.148209,2.461331,69.229318,108.22899,76.369367,0.848278
panel4,-39.395589,8.160427,16.368667,108.22899,77.016636,1.221379,-39.633990,-8.159890,16.216717,108.22899,61.090859,0.976875,-13.036450,3.130443,65.529983,108.22899,78.111841,0.930066
panel5,-42.484748,8.082001,14.220927,108.22899,76.467143,1.146475,-42.651122,-7.978157,14.430108,108.22899,61.958915,0.932345,-13.538288,3.346693,57.151595,108.22899,78.350691,1.127908


# ROUND & Output the files

In [14]:
#outputDf = outputDf.round(rounding_digits)
outputDf.to_excel(f'../data/{name}/output/processed_e.xlsx')
outputDf.head(10)


,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,sig_crit_shearLC1,sig_crit_biaxLC1,LC 1 RF,sigmaXXLC2,sigmaYYLC2,sigmaXYLC2,sig_crit_shearLC2,sig_crit_biaxLC2,LC 2 RF,sigmaXXLC3,sigmaYYLC3,sigmaXYLC3,sig_crit_shearLC3,sig_crit_biaxLC3,LC 3 RF
Component Name,,,,,,,,,,,,,,,,,,
panel1,-42.697845,8.136578,15.483408,108.22899,76.477754,1.131854,-42.916384,-7.917600,15.456793,108.22899,62.079210,0.923475,-14.302086,2.639952,61.968419,108.22899,76.285332,0.981501
panel2,-39.443552,8.188004,16.935501,108.22899,77.031279,1.214829,-39.430905,-8.217435,17.080579,108.22899,60.976510,0.974634,-12.703550,2.446052,67.486285,108.22899,76.541937,0.889847
panel3,-39.528903,8.051509,17.415443,108.22899,76.903741,1.205885,-39.804861,-8.170506,17.327509,108.22899,61.118929,0.966580,-13.148209,2.461331,69.229318,108.22899,76.369367,0.848278
panel4,-39.395589,8.160427,16.368667,108.22899,77.016636,1.221379,-39.633990,-8.159890,16.216717,108.22899,61.090859,0.976875,-13.036450,3.130443,65.529983,108.22899,78.111841,0.930066
panel5,-42.484748,8.082001,14.220927,108.22899,76.467143,1.146475,-42.651122,-7.978157,14.430108,108.22899,61.958915,0.932345,-13.538288,3.346693,57.151595,108.22899,78.350691,1.127908
